In [1]:
pip install langchain openai weaviate-client

  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.4 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached anyio-4.2.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached protobuf-4.25.2-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached charset_normalizer-3.3.2-cp38-cp38-macosx_10_9_x86_64.whl.metadata (33 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 5.3 MB/s eta 0:00:00
  Instal

In [7]:
pip install --upgrade pip

  Using cached https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 961.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.1/975.1 kB 8.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 12.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import creds

OPENAI_API_KEY = creds.OPENAI_TOKEN

import dotenv
dotenv.load_dotenv()

False

In [4]:
## Collect and Load Data

import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [16]:
## chunk document

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [18]:
len(chunks)

90

In [22]:
## embed and store the chunks

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    by_text = False
)

/Users/bobbycxy/Desktop/it_pays_to_learn/ML Topics/Natural Language Processing/RAG-examples/venv/lib/python3.8/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


embedded weaviate is already listening on port 8079


{"level":"info","msg":"Created shard langchain_96066d03a1c9480cbd18f28f85f2187b_sbmwoKJ95ntm in 6.532587ms","time":"2024-02-15T15:14:47+08:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-15T15:14:47+08:00","took":104003}
/Users/bobbycxy/Desktop/it_pays_to_learn/ML Topics/Natural Language Processing/RAG-examples/venv/lib/python3.8/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [23]:
vectorstore

In [24]:
## step 1: Retrieve
retriever = vectorstore.as_retriever()

In [25]:
retriever

VectorStoreRetriever(tags=['Weaviate', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.weaviate.Weaviate object at 0x7f871e49b1f0>)

In [26]:
## Step 2: Augment
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What did President Zelenskyy say in his speech?"#the president say about Justice Breyer"
rag_chain.invoke(query)

/Users/bobbycxy/Desktop/it_pays_to_learn/ML Topics/Natural Language Processing/RAG-examples/venv/lib/python3.8/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/bobbycxy/Desktop/it_pays_to_learn/ML Topics/Natural Language Processing/RAG-examples/venv/lib/python3.8/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'President Zelenskyy said in his speech to the European Parliament, "Light will win over darkness."'